<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Подготовка-окружения" data-toc-modified-id="Подготовка-окружения-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подготовка окружения</a></span></li><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Балансировка-данных" data-toc-modified-id="Балансировка-данных-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Балансировка данных</a></span></li><li><span><a href="#Создание-эмбеддинга-с-BERT" data-toc-modified-id="Создание-эмбеддинга-с-BERT-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Создание эмбеддинга с BERT</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Обучение-простой-модели" data-toc-modified-id="Обучение-простой-модели-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обучение простой модели</a></span></li><li><span><a href="#Обучение-на-сбалансированных-данных" data-toc-modified-id="Обучение-на-сбалансированных-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение на сбалансированных данных</a></span></li><li><span><a href="#Оптимизация-модели-с-помощью-Optuna" data-toc-modified-id="Оптимизация-модели-с-помощью-Optuna-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Оптимизация модели с помощью Optuna</a></span></li><li><span><a href="#Обучение-модели-на-эмбеддинге" data-toc-modified-id="Обучение-модели-на-эмбеддинге-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Обучение модели на эмбеддинге</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Подготовка окружения

Перед выполнением проекта подготовим окружение - загрузим необходимые библиотеки и данные.

In [2]:
!pip install optuna
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.2 MB/s 
     |████████████████████████████████| 209 kB 12.8 MB/s 
     |████████████████████████████████| 81 kB 11.1 MB/s 
     |████████████████████████████████| 78 kB 7.7 MB/s 
     |████████████████████████████████| 49 kB 7.0 MB/s 
     |████████████████████████████████| 146 kB 50.1 MB/s 
     |████████████████████████████████| 112 kB 56.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=4604e883f5c50c0b30095e3403af724f46136b9936551d5b42fe63e6ed1ebced
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.2 MB/s 
     |████████████████████████████████| 101 kB 10.3 M

In [3]:
 import pandas as pd
 import numpy as np

 import re
 import nltk
 nltk.download('wordnet')
 nltk.download('omw-1.4')
 from nltk.stem import WordNetLemmatizer
 from nltk.corpus import stopwords
 nltk.download('stopwords')
 from sklearn.feature_extraction.text import TfidfVectorizer

 import torch
 import transformers
 from tqdm import notebook

 from sklearn.model_selection import train_test_split
 from sklearn.utils import shuffle

 from sklearn.linear_model import LogisticRegression
 from sklearn.naive_bayes import MultinomialNB
 from sklearn.linear_model import SGDClassifier
 from sklearn.kernel_approximation import RBFSampler

 from sklearn.metrics import f1_score

 import optuna

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except FileNotFoundError:
    data = pd.read_csv('-') # Прямая ссылка на датасет удалена в связи с правилами публикации проектов

### Обзор данных

После загрузки библиотеки выведем имеющиеся данные на экран для ознакомления.

In [5]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


В имеющемся датасете около 160 тысяч строк с комментариями к товарам и метками "токсичного" или нет, текста.

Оценим сбалансированность распределения целевого показателя.

In [6]:
print('Доля негативных комментариев в датасете составляет ', '{:.2%}'.format(data['toxic'].mean()))

Доля негативных комментариев в датасете составляет  10.17%


Данные несбалансированы, 90% комментариев являются нейтральными или положительными. Токсичных комментариев всего 10% от всего датасета.

Пока нельзя однозначно сказать, повлияет ли на результат обучения модели дисбаланс, однако стоит проверить модели как на всем объеме данных, так и на сбалансированных данных.

Данные хранятся в виде чистого текста, для некоторых моделей их целесообразно преобразовать для дальнейшего обучения модели.


### Подготовка данных

Сперва удалим из текста наиболее часто встречающиеся символы и сокращения и заменим их пробелами, либо полными версиями слов.

In [7]:
def clean_text(text):
    text = text.lower()
    changing_dict = {"aren\'t":"are not","can\'t":"cannot","couldn\'t":"could not","didn\'t":"did not","doesn\'t":"does not","don\'t":"do not","hadn\'t":"had not","hasn\'t":"has not","haven\'t":"have not","he\'d":"he had","he\'ll":"he will","he\'s":"he is","i\'d":"i had","i\'ll":"i will","i\'m":"i am","i\'ve":"i have","isn\'t":"is not","it\'s":"it is","let\'s":"let us","mightn\'t":"might not","mustn\'t":"must not","shan\'t":"shall not","she\'d":"she had","she\'ll":"she will","she\'s":"she is","shouldn\'t":"should not","that\'s":"that is","there\'s":"there is","they\'d":"they had","they\'ll":"they will","they\'re":"they are","they\'ve":"they have","we\'d":"we had","we\'re":"we are","we\'ve":"we have","weren\'t":"were not","what\'ll":"what will","what\'re":"what are","what\'s":"what is","what\'ve":"what have","where\'s":"where is","who\'d":"who had","who\'ll":"who will","who\'re":"who are","who\'s":"who is","who\'ve":"who have","won\'t":"will not","wouldn\'t":"would not","you\'d":"you had","you\'ll":"you will","you\'re":"you are","you\'ve":"you have"}
    for key, value in changing_dict.items():
        text = text.replace(key, value)
    text = re.sub(r'\'s', '', text)
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = text.strip(' ')
    return text

In [9]:
corpus = data['text'].astype('U')
from tqdm import tqdm
tqdm.pandas(desc="progress bar!")

corpus = corpus.progress_apply(clean_text)

progress bar!: 100%|██████████| 159571/159571 [00:14<00:00, 11388.45it/s]


Лемматизируем имеющийся текст.

In [10]:
m = WordNetLemmatizer()

In [11]:
def lemmatize(text):
    text = text.split()
    return ' '.join([m.lemmatize(word) for word in text])

In [12]:
corpus = corpus.apply(lemmatize)

Разобъем датасет на тренировочную и тестовую выборки.

In [13]:
features_train, features_test, target_train, target_test = train_test_split(corpus, data['toxic'], test_size=.25)

Удалим стоп-слова из текста и вычислим tf idf для корпуса текста.

In [14]:
stopwords = set(stopwords.words('english'))

In [15]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [16]:
features_train = count_tf_idf.fit_transform(features_train)
features_test = count_tf_idf.transform(features_test)

### Балансировка данных

Как выяснилось в процессе изучения данных, выборка несбалансирована. Это может приводить к снижению качества обучения моделей. Исправим это для обучающей выборки. Сперва создадим выборку с уменьшением количества преобладающего класса.

In [17]:
def downsampling(features, target, fraction):
    features_zeros = features[target==0]
    features_ones = features[target==1]
    target_zeros = target[target==0]
    target_ones = target[target==1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=42)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=42)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=42)
    
    return features_downsampled, target_downsampled

In [18]:
features_downsampled = pd.DataFrame.sparse.from_spmatrix(features_train, index=target_train.index)

features_downsampled, target_downsampled = downsampling(features_downsampled, target_train, 0.1)

In [19]:
print(features_downsampled.shape)
print(target_downsampled.shape)
print('Доля негативных комментариев в сбалансированной обучающей выборке составляет ', '{:.2%}'.format(target_downsampled.mean()))

(22905, 133793)
(22905,)
Доля негативных комментариев в сбалансированной обучающей выборке составляет  53.05%


Несмотря на то, что данных с уменьшением выборки вполне достаточно для обучения модели, попробуем также обучить ее на расширенной выборке.

In [20]:
def upsampling(features, target, repeat):
    features_zeros = features[target==0]
    features_ones = features[target==1]
    target_zeros = target[target==0]
    target_ones = target[target==1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=42)
    
    return features_upsampled, target_upsampled

In [21]:
features_upsampled = pd.DataFrame.sparse.from_spmatrix(features_train, index=target_train.index)

features_upsampled, target_upsampled = upsampling(features_upsampled, target_train, 9)

In [22]:
print(features_upsampled.shape)
print(target_upsampled.shape)
print('Доля негативных комментариев в сбалансированной обучающей выборке составляет ', '{:.2%}'.format(target_upsampled.mean()))

(216894, 133793)
(216894,)
Доля негативных комментариев в сбалансированной обучающей выборке составляет  50.42%


Данные сбалансированы, создадим эмбеддинг наших текстов с помощью модели BERT.

### Создание эмбеддинга с BERT

Для оптимизации вычислений будем использовать только 2 тысячи случайных записей из датасета. Возьмем записи по индексам из сбалансированной, уменьшенной выборки, это гарантирует, что нам хватит данных для определения негативных комментариев.

In [23]:
bert_data = data.sample(n=2000, random_state=42)

 Загрузим предобученную модель DistilBert и токенизатор.

In [24]:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Токенизируем наши данные с помощью предобученной модели.

In [25]:
tokenized = bert_data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized = tokenized.apply(lambda x: x[:512])
max_len = tokenized.apply(len).max()

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

Token indices sequence length is longer than the specified maximum sequence length for this model (799 > 512). Running this sequence through the model will result in indexing errors


Разделим данные на батчи по 100 записей и преобразуем в эмбеддинги.

In [27]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/20 [00:00<?, ?it/s]

Создадим обучающую и тестовую выборки.

In [28]:
features_bert = np.concatenate(embeddings)
target_bert = bert_data['toxic']

features_train_bert, features_test_bert, target_train_bert, target_test_bert = train_test_split(features_bert, target_bert, test_size=.2, random_state=42)

Сбалансируем полученные данные путем увеличения выборки. Так как выборка и так слишком мала, балансировка через уменьшение нецелесообразна.

In [34]:
features_train_bert, target_train_bert = upsampling(pd.DataFrame(features_train_bert, index=target_train_bert.index), target_train_bert, 9)

In [35]:
print(features_train_bert.shape)
print(target_train_bert.shape)
print('Доля негативных комментариев в сбалансированной обучающей выборке составляет ', '{:.2%}'.format(target_train_bert.mean()))

(2952, 768)
(2952,)
Доля негативных комментариев в сбалансированной обучающей выборке составляет  51.52%


## Обучение

### Обучение простой модели

Обучим простую модель логистической регрессии.

In [39]:
model = LogisticRegression()

model.fit(features_train, target_train)
predictions = model.predict(features_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [40]:
f1_linear = f1_score(target_test, predictions)

In [41]:
print('Показатель F1 при использовании логистической регрессии равен ', {f1_linear})

Показатель F1 при использовании логистической регрессии равен  {0.7270011947431303}


Модель показывает значение метрики F1 близкое к необходимому, однако не дотягивает до установленной планки в 0.75.

Сравним качество работы логистической регрессии с другими простыми моделями.

In [42]:
models = (MultinomialNB, SGDClassifier)
for model in models:
    model = model()
    model.fit(features_train, target_train)
    predictions = model.predict(features_test)
    print(f'Показатель F1 при использовании {type(model).__name__} равен {f1_score(target_test, predictions)}.')

Показатель F1 при использовании MultinomialNB равен 0.35294117647058826.
Показатель F1 при использовании SGDClassifier равен 0.6259339199734352.


Модели наивного байеса и градиентного спуска показали себя хуже, чем логистическая регрессия.

### Обучение на сбалансированных данных

Сравним качество предсказания простой модели на несбалансированных данных с качеством на сбалансированных данных.

Сперва обучим модель на уменьшенных данных.

In [43]:
model = LogisticRegression()

model.fit(features_downsampled, target_downsampled)
predictions = model.predict(features_test)

In [44]:
f1_linear_downsampled = f1_score(target_test, predictions)

In [45]:
print('Показатель F1 при использовании логистической регрессии на уменьшенных данных равен ', {f1_linear_downsampled})

Показатель F1 при использовании логистической регрессии на уменьшенных данных равен  {0.6815298866038825}


И сравним с показателями модели на увеличенных данных.

In [46]:
model = LogisticRegression()

model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [47]:
f1_linear_upsampled = f1_score(target_test, predictions)

In [48]:
print('Показатель F1 при использовании логистической регрессии на увеличенных данных равен ', {f1_linear_upsampled})

Показатель F1 при использовании логистической регрессии на увеличенных данных равен  {0.7545265006035334}


Использование балансировки выборки с помощью увеличения позволило добиться наилучшего показателя F1.

### Оптимизация модели с помощью Optuna



Попробуем улучшить результаты логистической регрессии с помощью оптуны и найти наилучшие параметры обучения модели.

In [ ]:
def objective(trial):
    C = trial.suggest_loguniform("C", 1e-7, 100.0)
    solver = trial.suggest_categorical("solver", ("liblinear", "saga"))
    penalty = trial.suggest_categorical('penalty', ('l1', 'l2'))
    model = LogisticRegression(C=C, solver=solver, penalty=penalty)
    model.fit(features_train, target_train)
    predictions_opt = model.predict(features_test)
    
    return f1_score(target_test, predictions_opt)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2022-06-22 12:30:42,558] A new study created in memory with name: no-name-d87d738d-fcac-498e-b250-c2a6ee651e79
[I 2022-06-22 12:30:43,970] Trial 0 finished with value: 0.160147261849977 and parameters: {'C': 0.005240817312855058, 'solver': 'liblinear', 'penalty': 'l1'}. Best is trial 0 with value: 0.160147261849977.
[I 2022-06-22 12:30:49,968] Trial 1 finished with value: 0.7706396558901067 and parameters: {'C': 13.411778714891437, 'solver': 'liblinear', 'penalty': 'l2'}. Best is trial 1 with value: 0.7706396558901067.
[I 2022-06-22 12:32:45,748] Trial 2 finished with value: 0.0 and parameters: {'C': 5.529094240922278e-07, 'solver': 'saga', 'penalty': 'l2'}. Best is trial 1 with value: 0.7706396558901067.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
[I 2022-06-22 12:38:22,806] Trial 3 finished with value: 0.0 and parameters: {'C': 1.199966973014658e-0

В ходе оптимизиции наилучшим образом показала себя модель с показателями: C: 7.44, solver: saga, penalty: l1. Метрика F1 на тестовой выборке составила 0.7722, что выше необходимого уровня.

Обучим эту модель.

In [49]:
model = LogisticRegression(C=16.097240616234963, solver='saga', penalty='l1')

model.fit(features_train, target_train)
predictions = model.predict(features_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [50]:
f1_linear = f1_score(target_test, predictions)

In [51]:
print('Показатель F1 при использовании логистической регрессии с оптимизацией равен ', {f1_linear})

Показатель F1 при использовании логистической регрессии с оптимизацией равен  {0.7670693277310924}


После оптимизации получили неплохой показатель метрики, выше чем на сбалансированных данных.

### Обучение модели на эмбеддинге

Обучим модель на созданном с помощью BERT эмбеддинге.

In [52]:
model = LogisticRegression()

model.fit(features_train_bert, target_train_bert)
predictions = model.predict(features_test_bert)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [53]:
f1_bert = f1_score(target_test_bert, predictions)

In [54]:
print('Показатель F1 при использовании логистической регрессии и BERT равен ', {f1_bert})

Показатель F1 при использовании логистической регрессии и BERT равен  {0.6987951807228915}


Оптимизируем модель с BERT с помощью оптуны.

In [55]:
def objective(trial):
    C = trial.suggest_loguniform("C", 1e-7, 100.0)
    model = LogisticRegression(C=C, solver='saga', penalty='l1')
    model.fit(features_train_bert, target_train_bert)
    predictions_opt = model.predict(features_test_bert)
    
    return f1_score(target_test_bert, predictions_opt)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

[I 2022-06-23 19:27:06,670] A new study created in memory with name: no-name-aca160ce-72d8-4aeb-9c3f-367a50ad0252
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
[I 2022-06-23 19:27:11,180] Trial 0 finished with value: 0.6987951807228915 and parameters: {'C': 5.400459587476401}. Best is trial 0 with value: 0.6987951807228915.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
[I 2022-06-23 19:27:15,303] Trial 1 finished with value: 0.6236559139784946 and parameters: {'C': 0.10752401646695232}. Best is trial 0 with value: 0.6987951807228915.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
[I 2022-06-23 1

Модель с BERT показала худшие результаты, чем с использованием Tf idf. Лучшие результаты колеблются около 0.7.

## Выводы

В рамках проекта целесообразно использовать модель логистической регресии с параметрами C: 7.44, solver: saga, penalty: l1, которая показывает результат на тестовой выборке в 0.77.

Модель, обученная на сбалансированных, увеличенных данных, также показала себя на высоком уровне, однако время, которое тратит функция балансировки и модель для обучения на расширенной выборке, слишком велико - 10 минут на создание сбалансированной выборки и 30 минут на обучение модели на ней, в то время как обычная модель обучается гораздо быстрее и даже оптимизация с помощью оптуны выполняется быстрее.

Поиск по решетке значений выполнить не удалось, так как время перебора было слишком велико, а ограничения среды не позволяют выполнять такие длительные вычисления.

Модель BERT с логистической регрессией показывает весьма посредственный результат, вероятно в основном из-за несбалансированных данных и малого их числа - скорее всего модели не хватает достаточного количества положительных значений для создания информативного вектора, а большее количество наблюдений обработать не представляется возможным.